<a href="https://colab.research.google.com/github/PaulinaTarkowsk/sql_recap/blob/main/ploty_animations__plotly_shapes__dash_daq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
from google.colab import output, drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
population = pd.read_csv('/content/drive/My Drive/Dash/Dash_crossfiltering/population.csv')
new_books = pd.read_csv("/content/drive/My Drive/Dash/Dash_crossfiltering/new-books-per-million.csv")
data = pd.merge(new_books, population, how="inner", on=["Entity","Year"])
data = data[(data["Year"]>=1980) & (data["Year"]<=1996)]
countries_list = data[["Entity", "Year"]].groupby("Entity").count().sort_values(by="Year", ascending=False)
countries_list=countries_list[countries_list["Year"]==countries_list["Year"].max()]
data = pd.merge(data, countries_list, how="inner", on=["Entity"], suffixes=("","_z"))
data = data[data["Entity"]!="Russia"]

In [ ]:
fig = px.scatter(data_frame=data,
                 x="Population (historical estimates)",
                 y="Book titles per capita (Fink-Jensen 2015)",
                 size="Population (historical estimates)",
                 animation_frame="Year",
                 animation_group="Entity",
                 text="Entity",
                 hover_name="Entity",
                 size_max=30,
                 range_x=[1,100000000],
                 range_y=[1,5000],
                 template="plotly_white",
                 title="Population vs Book titles per capita in 1980-1996"
                 )

fig.update_traces(textposition="top right")
fig.update_yaxes(visible=True, showticklabels=True, title='# of Books')
fig.update_xaxes(visible=True, showticklabels=True, title='Population')
fig.update_layout(
    updatemenus=[
                 {
                     "type": "buttons",
                     "showactive": True,
                     "buttons": [{"label": "Start"}, {"label": "Stop"}]
                 }
    ]
)
fig

In [ ]:
def find_special_points(value, points):

    if value == points.max():
      special_point_name = "max"
    elif value == points.min():
      special_point_name = "min"
    elif value == points.median():
      special_point_name ="median"
    else:
      special_point_name = "not special"

    return special_point_name

In [ ]:
numbers = pd.DataFrame(np.random.randn(351), columns = ["sample",])
numbers["special_points"]=numbers["sample"].apply(lambda x: find_special_points(x,numbers["sample"]))

chart = px.scatter(
        numbers,
        x=numbers.index,
        y="sample",
        color="special_points",
        color_discrete_map={
            "max": "#800E13",
            "median": "#169873",
            "min": "#2C497F",
            "not special": "#E4DFDA",
                },
        render_mode="webgl",
        template='none',
        size=[10 for item in numbers["sample"]],
        title="Here're your points"
    )

cluster = [
           {
               "type": "rectangle",
               "x0": min(numbers.index),
               "y0": min(numbers["sample"]+0.5),
               "x1": max(numbers.index)/2,
               "y1": max(numbers["sample"]/2),
               "line": {"color": "DarkOrange", "width": 2, "dash": "dashdot"},
               "filcolor": "LightSkyBlue"
           }
]

chart.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(label="None",
                     method="relayout",
                     args=["shapes", []]),
                dict(label="Cluster",
                     method="relayout",
                     args=["shapes", cluster])
            ],
        )
    ]
)


In [ ]:
!pip install dash
!pip install dash_daq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.7/642.7 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dash_daq: filename=dash_daq-0.5.0-py3-none-any.whl size=669693 sha256=71d41ad54d53e626950ba713d54e6db21dfff290437b486bbce4bfd0ae8ccaca
  Stored in directory: /root/.cache/pip/wheels/75/14/1b/208d09d5e239391048bdc167759977b41ba65a3d4063aebf6b
Successfully built dash_daq


In [ ]:
from dash import Dash, dcc, html, callback, Input, Output
import dash_daq as daq
import threading

In [ ]:
app = Dash()

app.layout = html.Div([
    daq.BooleanSwitch(id='bswitch', on=False),
    daq.Knob(id='knob'),
    html.Div(id='bswitch-info'),
    daq.Gauge(id='gauge', label="Gauge", value=8),
    daq.GraduatedBar(id='gbar', label="Graduated bar",value=6),
    html.Div([dcc.Slider(id='slider', min=0, max=10, step=1, value=3)], style={"width": "100%"}),
],
         style={
             "display": "flex",
             "flex-direction": "column",
             "justify-content": "center",
             "align-items": 'center',
             "align-content": "center",
             "height": "100%",
             "width": "75%",
             "fontFamily": "verdana",
             "color": "#444"
             }
)

@app.callback(
    Output('bswitch-info', 'children'),
    Input('bswitch', 'on'),
    Input('knob', 'value')
)
def update_output1(on, value):
    return ('Switched {}.'.format(on)
    + ', the knob is set to {}.'.format(value))

@app.callback(
    Output('gauge', 'value'),
    Output('gbar', 'value'),
    Output('bswitch', 'on'),
    Input('slider', 'value'))
def update_output2(value):
    if value < 5:
      on = False
    else:
      on = True
    return value, value, on


In [ ]:
thread = threading.Thread(target=app.run_server)
thread.start()
output.serve_kernel_port_as_window(port=8050)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>